In [1]:
import os
from shutil import copy
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import json

## Read data

In [2]:
with open ('../input/caries-json-annotations/annot.json') as file:
    annot = json.load(file)

In [3]:
annotations = pd.DataFrame(annot['annotations'])
images = pd.DataFrame(annot['images']).rename(columns={'id':'image_id'})

data = annotations.merge(images, on='image_id')

data = data[['bbox', 'category_id', 'file_name']]
data['x_from'] = data['bbox'].apply(lambda x: x[0])
data['y_from'] = data['bbox'].apply(lambda x: x[1])
data['width'] = data['bbox'].apply(lambda x: x[2])
data['height'] = data['bbox'].apply(lambda x: x[3])
data['class'] = data['category_id'].replace({1:'teeth', 2:'caries'})
data = data[(data['x_from'] > 0) & (data['y_from'] > 0) & (data['width'] > 0) & (data['height'] > 0)]

data = data[data['file_name'].isin(os.listdir('../input/clear-teeth-dataset/clear_dataset'))]
data = data.reset_index(drop=True)
data

,bbox,category_id,file_name,x_from,y_from,width,height,class
0,"[120, 51, 132, 44]",1,582.jpg,120,51,132,44,teeth
1,"[79, 64, 36, 33]",1,582.jpg,79,64,36,33,teeth
2,"[52, 91, 39, 55]",1,582.jpg,52,91,39,55,teeth
3,"[60, 201, 44, 55]",1,582.jpg,60,201,44,55,teeth
4,"[233, 198, 23, 58]",1,582.jpg,233,198,23,58,teeth
...,...,...,...,...,...,...,...,...
4362,"[1, 47, 43, 32]",1,1186.jpg,1,47,43,32,teeth
4363,"[6, 31, 44, 18]",1,1186.jpg,6,31,44,18,teeth
4364,"[236, 33, 20, 46]",1,1186.jpg,236,33,20,46,teeth
4365,"[51, 1, 79, 25]",1,1186.jpg,51,1,79,25,teeth


## Data preparation

In [5]:
filenames = data.file_name.unique()
np.random.seed(0)
np.random.shuffle(filenames)

In [6]:
len(filenames)

488

In [7]:
train_files = filenames[:-60]
val_files = filenames[-60:-10]
test_files = filenames[-10:]

In [8]:
len(train_files)

428

## Split data to train, val, test

In [12]:
train = data[data.file_name.isin(train_files)]
train.to_csv('train.csv', index=False)
os.mkdir('train')
for i in train_files:
    copy(f'../input/teeth-caries-children/all/{i}', f'train/{i}')
    
val = data[data.file_name.isin(val_files)]
val.to_csv('val.csv', index=False)
os.mkdir('val')
for i in val_files:
    copy(f'../input/teeth-caries-children/all/{i}', f'val/{i}')

test = data[data.file_name.isin(test_files)]
test.to_csv('test.csv', index=False)
os.mkdir('test')
for i in test_files:
    copy(f'../input/teeth-caries-children/all/{i}', f'test/{i}')

## Install libs, requirements

In [13]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [14]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1


In [15]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 733.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0
    Uninstalling torchaudio-0.11.0:
      Successfully uninstalled torchaudio-0.11.0


In [16]:
!pip show torch

Name: torch
Version: 1.10.0+cu113
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.7/site-packages
Requires: typing-extensions
Required-by: accelerate, allennlp, catalyst, easyocr, fairscale, fastai, kornia, pytorch-ignite, pytorch-lightning, torchaudio, torchmetrics, torchtext, torchvision


In [17]:
# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 MB 7.3 MB/s eta 0:00:00


In [18]:
# Install mmdetection
!pip install mmdet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - \ | / done
  Preparing metadata (pyproject.toml) ... - \ | / done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=370081 sha256=f36ef1908e81e95d37a60bc7481f5c4d40142b2a9ffca92be62d03d98b34c76d
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools


In [19]:
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 24969, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 24969 (delta 3), reused 6 (delta 1), pack-reused 24959
Receiving objects: 100% (24969/24969), 37.76 MiB | 27.39 MiB/s, done.
Resolving deltas: 100% (17503/17503), done.
/kaggle/working/mmdetection
Obtaining file:///kaggle/working/mmdetection
  Preparing metadata (setup.py) ... - \ done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.25.0
    Uninstalling mmdet-2.25.0:
      Successfully uninstalled mmdet-2.25.0
  Running setup.py develop for mmdet


## Check the installation

In [20]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

!ls

1.10.0+cu113 True
2.25.0
1.5.3
11.1
GCC 7.3
CITATION.cff  README_zh-CN.md  docs		pytest.ini	  setup.cfg
LICENSE       configs	       mmdet		requirements	  setup.py
MANIFEST.in   demo	       mmdet.egg-info	requirements.txt  tests
README.md     docker	       model-index.yml	resources	  tools


## Download model

In [21]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth \
    -O checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth

--2022-06-24 20:56:53--  https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277858723 (265M) [application/octet-stream]
Saving to: ‘checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth’

checkpoints/detecto 100%[===================>] 264.99M  7.76MB/s    in 31s     

2022-06-24 20:57:25 (8.58 MB/s) - ‘checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth’ saved [277858723/277858723]



## Custom TeethDataset for training

In [27]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module()
class TeethDataset(CustomDataset):

    CLASSES = ['teeth', 'caries']

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        anno = pd.read_csv(ann_file)
        image_list = anno.file_name.unique()
    
        data_infos = []
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}', width=width, height=height)
    
            
            image_anno = anno[anno.file_name == image_id]
            bbox_names = anno['class']
            bboxes = [[line.x_from, line.y_from, line.x_from+line.width, line.y_from+line.height] for line in image_anno.itertuples()]
    
            gt_bboxes = []
            gt_labels = []
    
            for bbox_name, bbox in zip(bbox_names, bboxes):
                gt_labels.append(cat2label[bbox_name])
                gt_bboxes.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

## Create the training config

In [28]:
from mmcv import Config
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py')

In [29]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'TeethDataset'
cfg.data_root = '/kaggle/working/'

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 6
cfg.imgs_per_gpu = 4
cfg.batch_size = 4

cfg.data.test.type = 'TeethDataset'
cfg.data.test.data_root = '/kaggle/working/'
cfg.data.test.ann_file = 'test.csv'
cfg.data.test.img_prefix = 'test'

cfg.data.train.type = 'TeethDataset'
cfg.data.train.data_root = '/kaggle/working/'
cfg.data.train.ann_file = 'train.csv'
cfg.data.train.img_prefix = 'train'

cfg.data.val.type = 'TeethDataset'
cfg.data.val.data_root = '/kaggle/working/'
cfg.data.val.ann_file = 'val.csv'
cfg.data.val.img_prefix = 'val'

for i in cfg.model.roi_head.bbox_head:
    i['num_classes'] = 2
cfg.load_from = 'checkpoints/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth'

cfg.work_dir = '/kaggle/working/model'

cfg.optimizer.lr = 0.001
cfg.lr_config.warmup = None
# cfg.lr_config.min_lr = 3e-4
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

cfg.device = "cuda"

cfg.runner.max_epochs = 10

## Train datector

In [30]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, inference_detector, init_detector, show_result_pyplot


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/kaggle/working/mmdetection/mmdet/datasets/custom.py:180: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This Data

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 50/50, 10.4 task/s, elapsed: 5s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-06-24 20:58:59,355 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| teeth  | 388 | 1769 | 0.876  | 0.702 |
| caries | 6   | 0    | 0.000  | 0.000 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.351 |
+--------+-----+------+--------+-------+
2022-06-24 20:58:59,359 - mmdet - INFO - Epoch(val) [1][50]	AP50: 0.3510, mAP: 0.3512
2022-06-24 20:59:09,402 - mmdet - INFO - Epoch [2][10/107]	lr: 1.000e-03, eta: 0:10:42, time: 0.995, data_time: 0.306, memory: 4749, loss_rpn_cls: 0.0267, loss_rpn_bbox: 0.0271, s0.loss_cls: 0.2105, s0.acc: 90.9766, s0.loss_bbox: 0.2004, s1.loss_cls: 0.1046, s1.acc: 91.3548, s1.loss_bbox: 0.2385, s2.loss_cls: 0.0644, s2.acc: 88.1990, s2.loss_bbox: 0.1331, loss: 1.0053
2022-06-24 20:59:15,873 - mmdet - INFO - Epoch [2][20/107]	lr: 1.000e-03, eta: 0:10:33, time: 0.647, data_time: 0.019, memory: 4749, loss_rpn_cls: 0.0345, loss_rpn_bbox: 0.0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 50/50, 10.1 task/s, elapsed: 5s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-06-24 21:00:19,790 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| teeth  | 388 | 1206 | 0.884  | 0.747 |
| caries | 6   | 0    | 0.000  | 0.000 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.373 |
+--------+-----+------+--------+-------+
2022-06-24 21:00:19,794 - mmdet - INFO - Epoch(val) [2][50]	AP50: 0.3730, mAP: 0.3733
2022-06-24 21:00:29,540 - mmdet - INFO - Epoch [3][10/107]	lr: 1.000e-03, eta: 0:09:15, time: 0.965, data_time: 0.286, memory: 4749, loss_rpn_cls: 0.0155, loss_rpn_bbox: 0.0180, s0.loss_cls: 0.1913, s0.acc: 91.9092, s0.loss_bbox: 0.1718, s1.loss_cls: 0.0919, s1.acc: 92.4172, s1.loss_bbox: 0.2057, s2.loss_cls: 0.0530, s2.acc: 90.9392, s2.loss_bbox: 0.1394, loss: 0.8866
2022-06-24 21:00:36,034 - mmdet - INFO - Epoch [3][20/107]	lr: 1.000e-03, eta: 0:09:08, time: 0.649, data_time: 0.021, memory: 4749, loss_rpn_cls: 0.0192, loss_rpn_bbox: 0.0

...

2022-06-24 21:08:17,689 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| teeth  | 388 | 788  | 0.874  | 0.759 |
| caries | 6   | 0    | 0.000  | 0.000 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.379 |
+--------+-----+------+--------+-------+
2022-06-24 21:08:17,693 - mmdet - INFO - Epoch(val) [8][50]	AP50: 0.3790, mAP: 0.3793
2022-06-24 21:08:27,811 - mmdet - INFO - Epoch [9][10/107]	lr: 1.000e-04, eta: 0:02:11, time: 1.002, data_time: 0.336, memory: 4749, loss_rpn_cls: 0.0091, loss_rpn_bbox: 0.0183, s0.loss_cls: 0.1504, s0.acc: 93.4912, s0.loss_bbox: 0.1288, s1.loss_cls: 0.0648, s1.acc: 94.3926, s1.loss_bbox: 0.1742, s2.loss_cls: 0.0397, s2.acc: 93.2976, s2.loss_bbox: 0.1269, loss: 0.7123
2022-06-24 21:08:34,219 - mmdet - INFO - Epoch [9][20/107]	lr: 1.000e-04, eta: 0:02:04, time: 0.641, data_time: 0.023, memory: 4749, loss_rpn_cls: 0.0087, loss_rpn_bbox: 0.0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 50/50, 10.3 task/s, elapsed: 5s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-06-24 21:09:37,450 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| teeth  | 388 | 767  | 0.879  | 0.769 |
| caries | 6   | 0    | 0.000  | 0.000 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.384 |
+--------+-----+------+--------+-------+
2022-06-24 21:09:37,454 - mmdet - INFO - Epoch(val) [9][50]	AP50: 0.3840, mAP: 0.3843
2022-06-24 21:09:47,086 - mmdet - INFO - Epoch [10][10/107]	lr: 1.000e-04, eta: 0:01:02, time: 0.954, data_time: 0.279, memory: 4749, loss_rpn_cls: 0.0098, loss_rpn_bbox: 0.0218, s0.loss_cls: 0.1503, s0.acc: 93.6328, s0.loss_bbox: 0.1320, s1.loss_cls: 0.0655, s1.acc: 94.6909, s1.loss_bbox: 0.1736, s2.loss_cls: 0.0382, s2.acc: 93.4876, s2.loss_bbox: 0.1227, loss: 0.7139
2022-06-24 21:09:53,492 - mmdet - INFO - Epoch [10][20/107]	lr: 1.000e-04, eta: 0:00:55, time: 0.641, data_time: 0.020, memory: 4749, loss_rpn_cls: 0.0079, loss_rpn_bbox: 0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 50/50, 10.5 task/s, elapsed: 5s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-06-24 21:10:56,157 - mmdet - INFO - 
+--------+-----+------+--------+-------+
| class  | gts | dets | recall | ap    |
+--------+-----+------+--------+-------+
| teeth  | 388 | 738  | 0.876  | 0.767 |
| caries | 6   | 0    | 0.000  | 0.000 |
+--------+-----+------+--------+-------+
| mAP    |     |      |        | 0.384 |
+--------+-----+------+--------+-------+
2022-06-24 21:10:56,160 - mmdet - INFO - Epoch(val) [10][50]	AP50: 0.3840, mAP: 0.3836


## Check model results

In [31]:
model.cfg = cfg

In [32]:
model.cfg.dump("/kaggle/working/rcnn_teeth_v3_50ep.py")